In [1]:
using ParallelTemperingMonteCarlo, StaticArrays, DelimitedFiles

Broadly we need to save:

-Params which do not change

-Configs (the BC may change?)

-Results

Beginning with the static parameters which do not change:

In [2]:
n_atoms = 13
ti = 12.
tf = 16.
n_traj = 5
temp = TempGrid{n_traj}(ti,tf) 
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles
displ_atom = 0.1 # Angstrom
n_adjust = 100
max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]
mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)
c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

a=[0.0005742,-0.4032,-0.2101,-0.0595,0.0606,0.1608]
b=[-0.01336,-0.02005,-0.1051,-0.1268,-0.1405,-0.1751]
c1=[-0.1132,-1.5012,35.6955,-268.7494,729.7605,-583.4203]
potB = ELJPotentialB{6}(a,b,c1)

ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)


MoveStrategy{13, NVT}(NVT(13, 13, 0), ["atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove"])

Now the configurations, we follow the xyz format. 

In [3]:
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

#binding sphere
bc_ne13 = SphericalBC(radius=5.32*AtoBohr) 

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

start_config = Config(pos_ne13, bc_ne13)

Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.841053896189476e-7]], SphericalBC{Float64}(101.06969058367278))

Now we can write individual configs, we need the overarching function. Most likely a for i in eachindex states, in this way we have a config.$i 

In [4]:
# coeffs= Vector(potinfovec[1,2:end])
# checkpot = ELJPotential(coeffs)
evtohartree = 0.0367493
nmtobohr = 18.8973
#parameters taken from L Vocadlo etal J Chem Phys V120N6 2004
n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

eampot = EmbeddedAtomPotential(n,m,ϵ,C,a)


EmbeddedAtomPotential(8.482, 4.692, 356.5912625675561, 1.4313533476465632)

In [5]:
states,results = ptmc_run!(mc_params,temp,start_config,pot,ensemble;rdfsave=false,save=1000)

params set
equilibration complete
MC loop done.
[82.54360001292939, 78.91187288572053, 62.325812737284295, 48.15047713636543, 38.895349658155276]
done


(MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}[MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}(12.0, 26314.5821052581, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-0.9038085846702818, -8.84959196736848, 1.8600008663566103], [3.336839593594328, -0.9834693157079114, 5.69404666047846], [1.3504127557314034, -3.3712075883827115, 1.2837181792687773], [-2.3278149963797348, 0.3026514255980708, 3.7806231633811582], [6.675292571569033, -2.9708785288004185, 0.8886100714286063], [-1.606465042409246, -4.852407878098309, 6.059808898709351], [4.000620752894418, -3.297664271032903, -4.200117765193415], [-2.2258882373416387, -0.3708407913054446, -2.367070101785326], [-0.8017071064732361, -5.833874137542632, -3.243427211448866], [4.288149795779289, -7.863795469683128, -1.0038108967708224], [4.238890639110813, -7.513156064684622, 4.456426005496906], [2.717632319419582, 2.0180872120

In [11]:
states,results = ptmc_run!(true;rdfsave=false)

params set
equilibration complete
MC loop done.
[0.0, 0.0, 0.0, 0.0, 0.0]
done


(MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}[MCState{Float64, 13, SphericalBC{Float64}, DimerPotentialVariables{Float64}, NVTVariables{Float64}}(12.0, 26314.5821052581, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-0.9038085846702818, -8.84959196736848, 1.8600008663566103], [3.336839593594328, -0.9834693157079114, 5.69404666047846], [1.3504127557314034, -3.3712075883827115, 1.2837181792687773], [-2.3278149963797348, 0.3026514255980708, 3.7806231633811582], [6.675292571569033, -2.9708785288004185, 0.8886100714286063], [-1.606465042409246, -4.852407878098309, 6.059808898709351], [4.000620752894418, -3.297664271032903, -4.200117765193415], [-2.2258882373416387, -0.3708407913054446, -2.367070101785326], [-0.8017071064732361, -5.833874137542632, -3.243427211448866], [4.288149795779289, -7.863795469683128, -1.0038108967708224], [4.238890639110813, -7.513156064684622, 4.456426005496906], [2.717632319419582, 2.0180872120

In [12]:
states[1].count_atom

2-element Vector{Int64}:
 662122
      0